In [1]:
from transformers import BartForConditionalGeneration, BartTokenizer

In [2]:
# Load the model and tokenizer
model = BartForConditionalGeneration.from_pretrained(
    'facebook/bart-large-cnn')
tokenizer = BartTokenizer.from_pretrained(
    'facebook/bart-large-cnn')

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

C:\Users\mshos\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\mshos\.cache\huggingface\hub\models--facebook--bart-large-cnn. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

In [3]:
def summarize(text, maxSummarylength=500):
    # Encode the text and summarize
    inputs = tokenizer.encode("summarize: " +
                              text,
                              return_tensors="pt",
                              max_length=1024, truncation=True)
    summary_ids = model.generate(inputs, max_length=maxSummarylength,
                                 min_length=int(maxSummarylength/5),
                                 length_penalty=10.0,
                                 num_beams=4, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

In [4]:
def split_text_into_pieces(text,
                           max_tokens=900,
                           overlapPercent=10):
    # Tokenize the text
    tokens = tokenizer.tokenize(text)

    # Calculate the overlap in tokens
    overlap_tokens = int(max_tokens * overlapPercent / 100)

    # Split the tokens into chunks of size
    # max_tokens with overlap
    pieces = [tokens[i:i + max_tokens]
              for i in range(0, len(tokens),
                             max_tokens - overlap_tokens)]

    # Convert the token pieces back into text
    text_pieces = [tokenizer.decode(
        tokenizer.convert_tokens_to_ids(piece),
        skip_special_tokens=True) for piece in pieces]

    return text_pieces


In [5]:

def recursive_summarize(text, max_length=200, recursionLevel=0):
    recursionLevel=recursionLevel+1
    print("######### Recursion level: ",
          recursionLevel,"\n\n######### ")
    tokens = tokenizer.tokenize(text)
    expectedCountOfChunks = len(tokens)/max_length
    max_length=int(len(tokens)/expectedCountOfChunks)+2

    # Break the text into pieces of max_length
    pieces = split_text_into_pieces(text, max_tokens=max_length)

    print("Number of pieces: ", len(pieces))
    # Summarize each piece
    summaries=[]
    k=0
    for k in range(0, len(pieces)):
        piece=pieces[k]
        print("****************************************************")
        print("Piece:",(k+1)," out of ", len(pieces), "pieces")
        print(piece, "\n")
        summary =summarize(piece, maxSummarylength=max_length/3*2)
        print("SUMNMARY: ", summary)
        summaries.append(summary)
        print("****************************************************")

    concatenated_summary = ' '.join(summaries)

    tokens = tokenizer.tokenize(concatenated_summary)

    if len(tokens) > max_length:
        # If the concatenated_summary is too long, repeat the process
        print("############# GOING RECURSIVE ##############")
        return recursive_summarize(concatenated_summary,
                                   max_length=max_length,
                                   recursionLevel=recursionLevel)
    else:
      # Concatenate the summaries and summarize again
        final_summary=concatenated_summary
        if len(pieces)>1:
            final_summary = summarize(concatenated_summary,
                                  maxSummarylength=max_length)
        return final_summary

In [6]:
# Example usage
text = '''Once upon a time, in a land far, far away, there lived a young girl named Amelia. Amelia lived in a small cottage at the edge of a dense forest with her mother, father, and younger brother, Max. The forest was a magical place, filled with all sorts of creatures, both friendly and fearsome. Amelia and Max loved to explore the forest, and they had many adventures there.

One day, while Amelia and Max were exploring a particularly dense part of the forest, they stumbled upon a hidden cave. The cave was tucked away behind a thicket of bushes and was almost invisible unless you knew where to look. Amelia and Max were curious and decided to venture inside.
a
The inside of the cave was dark and damp, but Amelia and Max were not afraid. They had brought a lantern with them, and Amelia lit it as they ventured further into the cave. As they walked deeper into the cave, they noticed strange markings on the walls. They were unlike anything they had ever seen before.

Amelia and Max continued to explore the cave, and as they ventured deeper, they came across a large chamber. In the center of the chamber was a pedestal with a small, glowing object resting on it. Amelia approached the pedestal and picked up the object. It was a small, round stone that emitted a soft, warm light.

As soon as Amelia picked up the stone, the ground began to tremble, and the walls of the cave started to shake. Amelia and Max were frightened and ran out of the cave as fast as they could. When they reached the entrance of the cave, they turned around and saw that the cave had collapsed behind them.

Amelia and Max were relieved to have escaped the cave, but they were also curious about the stone Amelia had found. They decided to take it home with them and show it to their parents.

When they arrived home, their parents were surprised to see them with the stone. Amelia and Max explained how they had found it in the cave and how the cave had collapsed after they took the stone. Their parents were worried but also intrigued by the stone.

Amelia's father, a scholar and historian, examined the stone carefully. He recognized the markings on the stone as an ancient language that had long been forgotten. He believed that the stone was a key to unlocking a hidden treasure that was said to be buried deep within the forest.

Excited by this discovery, Amelia and Max decided to go on a quest to find the hidden treasure. Their parents were worried about the dangers they might encounter along the way, but they knew that Amelia and Max were resourceful and capable.

The next morning, Amelia and Max set out on their adventure. They packed a bag with food, water, and other supplies, and headed into the forest. They followed a map that their father had given them, which he believed would lead them to the hidden treasure.

As they ventured deeper into the forest, Amelia and Max encountered all sorts of creatures, some friendly and some not so friendly. They came across a river that was too wide to cross, so they built a raft and sailed across. They climbed steep hills, crossed narrow bridges, and navigated through dense thickets.

After several days of traveling, Amelia and Max arrived at the location marked on the map. It was a small clearing in the middle of the forest, surrounded by tall trees. In the center of the clearing was a large rock with a hole in the center.

Amelia and Max approached the rock and examined the hole. It was just large enough for the stone that Amelia had found in the cave. Amelia placed the stone into the hole, and as she did so, the ground began to tremble again.

This time, however, the trembling did not stop. The ground cracked open, and a hidden staircase appeared. Amelia and Max were excited and a little bit nervous as they descended the staircase.

At the bottom of the staircase was a large chamber, similar to the one they had found in the cave. In the center of the chamber was a pedestal with a chest resting on it. Amelia and Max approached the chest and opened it.

Inside the chest was a collection of gold coins, jewels, and other valuable items. Amelia and Max were thrilled to have found the hidden treasure. They carefully packed the treasure into their bags and began the journey home.

When they arrived home, their parents were relieved to see them safe and sound. Amelia and Max showed them the treasure they had found, and their parents were amazed. They were proud of their children for their bravery and resourcefulness.

Amelia and Max were heroes in their village, and the story of their adventure became legendary. They had found the hidden treasure and returned home safely, proving that with courage and determination, anything is possible.'''


In [7]:
final_summary = recursive_summarize(text)
print("\n%%%%%%%%%%%%%%%%%%%%%\n")
print("Final summary:", final_summary)

######### Recursion level:  1 

######### 
Number of pieces:  6
****************************************************
Piece: 1  out of  6 pieces
Once upon a time, in a land far, far away, there lived a young girl named Amelia. Amelia lived in a small cottage at the edge of a dense forest with her mother, father, and younger brother, Max. The forest was a magical place, filled with all sorts of creatures, both friendly and fearsome. Amelia and Max loved to explore the forest, and they had many adventures there.

One day, while Amelia and Max were exploring a particularly dense part of the forest, they stumbled upon a hidden cave. The cave was tucked away behind a thicket of bushes and was almost invisible unless you knew where to look. Amelia and Max were curious and decided to venture inside.
a
The inside of the cave was dark and damp, but Amelia and Max were not afraid. They had brought a lantern with them, and Amelia lit it as they ventured further into the cave. As they walked deeper